In [31]:
#Steps to loading file into Tabelue
#1. start-all in 
#2. Run MongoSQLD
#3. run below code in cmd 
#pyspark --conf "spark.mongodb.input.uri=mongodb://127.0.0.1/dbase.quakes?readPreference=primaryPrefered" --conf "spark.mongodb.output.uri=mongodb://127.0.0.1/dbase.quakes" --packages org.mongodb.spark:mongo-spark-connector_2.12:2.4.1
#4. put csv in bigdata folder. Open Cmd, type; hdfs dfs -ls /hdfs dfs -mkdir /class hdfs dfs -copyFromLocal C:/bigdata_class/car_data.csv /class hdfs dfs -ls 
#5. Upload this Jupyter note, run all the code
#6. Refresh NoSQL Mongodb Test connection with ODBC 
#7. Open in tableu under servers\

In [32]:
import pyspark

In [33]:
from pyspark.sql import SparkSession

In [34]:
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [35]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder\
        .master('local[2]')\
        .appName('quake_etl')\
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
        .getOrCreate()

In [36]:
#Code above reused from Material 12's spark-mongodb Notebook

In [37]:
#loading our data set from the big_data folder, needs Hadoop commands to load 
df_load = spark.read.csv('/class/bcar.csv',inferSchema=True, header=True)
df_load.printSchema()

root
 |-- #: integer (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Manufacturer's Suggested Retail Price: string (nullable = true)
 |-- Price - MSRP: string (nullable = true)
 |-- Brand: string (nullable = true)



In [38]:
df_car = df_load

In [39]:
df_car.show(5)

+---+--------------------+----+------+-------------+--------+-------------------------------------+------------+-----+
|  #|               Model|Year|Status|      Mileage|   Price|Manufacturer's Suggested Retail Price|Price - MSRP|Brand|
+---+--------------------+----+------+-------------+--------+-------------------------------------+------------+-----+
|  0|2023 Acura RDX A-...|2023|   New|Not available|50895.00|                        Not specified|     #VALUE!|Acura|
|  1|2023 Acura TLX Ty...|2023|   New|Not available|57745.00|                        Not specified|     #VALUE!|Acura|
|  2|2023 Acura TLX Ty...|2023|   New|Not available|57545.00|                        Not specified|     #VALUE!|Acura|
|  3|2023 Acura TLX Ty...|2023|   New|Not available|57745.00|                        Not specified|     #VALUE!|Acura|
|  4|2023 Acura TLX A-...|2023|   New|Not available|47995.00|                        Not specified|     #VALUE!|Acura|
+---+--------------------+----+------+----------

In [40]:
df_car.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri', 'mongodb://127.0.0.1:27017/dbase.quakes').save()
